In [1]:
### STEP 1. Preamble
import coloredlogs
import ipdb
import json
import logging
import os
import pylatex
import pylatexenc
from pylatexenc import latexencode  # line is necessary
import re
import subprocess
import tkinter
import tkinter.filedialog

verbose = True

### STEP 2. Set up logger
log = logging.getLogger(__name__)
if verbose:
    coloredlogs.install(fmt='%(asctime)s [%(levelname)s] %(message)s', level=logging.DEBUG, logger=log)
else:
    coloredlogs.install(fmt='%(asctime)s [%(levelname)s] %(message)s', level=logging.INFO, logger=log)

In [2]:
### STEP 3. Read JSON infile
tkinter.Tk().withdraw() ## Prevent root window
in_fn = tkinter.filedialog.askopenfilename()

output_fn = in_fn.strip(".json")

with open(in_fn, 'r') as file:
    masterDict = json.load(file)   

In [ ]:
### STEP 4. Writing results to output LaTeX document
action = "writing output to LaTeX file"
log.info("%s" % action)
geometry_options = {'tmargin': '2cm', 'lmargin': '2cm', 'bmargin': '2cm', 'rmargin': '2cm'}
doc = pylatex.Document(geometry_options=geometry_options)#, inputenc = 'utf8x')#, fontenc='T1')
with doc.create(pylatex.Section('Results of literature mining')):
    
    for uid, article in masterDict.items():
        if uid == 'pubmed_query':
            continue
        if article['article_result_paragraphs']:
            with doc.create(pylatex.Subsection(article['article_title'])):
                doc.append('Year: %s | PubMed ID: %s | PubMedCentral ID: %s\n\n' % (article['article_publdate']['Year'], article['article_pmid'], article['article_pmcid']))
                doc.append('Keywords: %s\n\n' % (article['article_keywords']))
                doc.append('MeSH: %s\n\n' % (article['article_mesh']))
                
                # Fuse all paragraphs into single string
                main_text = '\n\n'.join(article['article_result_paragraphs'])

                # Conducting case-insensitive string replacements
                for gene in article['found_genes']:
                    if gene.casefold() in main_text.casefold():
                        gene_repl = '___' + gene + '|||'
                        main_text = re.sub(gene, gene_repl, main_text, flags=re.IGNORECASE)
                for toxin in article['found_toxins']:
                    if toxin.casefold() in main_text.casefold():
                        toxin_repl = '$$$' + toxin + '!!!'
                        main_text = re.sub(toxin, toxin_repl, main_text, flags=re.IGNORECASE)
                main_text = main_text.replace(" gene ", ' +++'+"gene"+'=== ')
                main_text = main_text.replace(" gene,", ' +++'+"gene"+'===,')
                main_text = main_text.replace(" gene.", ' +++'+"gene"+'===.')
                    
                # Enforce a character encoding, if any
                #main_text = main_text.encode('utf8')
                main_text = pylatexenc.latexencode.unicode_to_latex(main_text)
                doc.append(pylatex.utils.NoEscape(main_text))  ## NoEscape tells pylatex your string is already encoded as latex and not to encode it again

                #doc.append(main_text)

#doc.packages.append(pylatex.Package('xcolor'))compiling LaTeX file to PDF
doc.preamble.append(pylatex.Command('usepackage', 'helvet'))
doc.preamble.append(pylatex.NoEscape('\\usepackage[rgb,dvipsnames]{xcolor}'))
doc.preamble.append(pylatex.NoEscape('\\usepackage[utf8]{inputenc}'))
doc.preamble.append(pylatex.NoEscape(r'\renewcommand{\familydefault}{\sfdefault}'))

NameError: name 'log' is not defined